In [69]:

{-# LANGUAGE OverloadedStrings #-}
import Control.Applicative
import Data.Aeson
import Data.Text (Text)

-- Same type as before, but without the Generic instance.
data Person = Person { name :: Text, age :: Int } deriving Show

-- We expect a JSON object, so we fail at any non-Object value.
instance FromJSON Person where
    parseJSON (Object v) = Person <$> v .: "name" <*> v .: "age"


instance ToJSON Person where
    toJSON (Person name age) = object ["name" .= name, "age" .= age]

-- The main function is unchanged from before.

e = encode (Person { name = "Joe", age = 12 })
e
decode e :: Maybe Person

"{\"age\":12,\"name\":\"Joe\"}"

Just (Person {name = "Joe", age = 12})

In [19]:
kafka    = "[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"
redshift = "[[\"odbc_driver\",\"/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so\"],[\"db_node\",\"us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com\"],[\"db_port\",\"5439\"],[\"db_name\",\"redshift_hvr\"]]"

In [3]:
decode red :: Maybe Value
decode kafka :: Maybe Value

Just (Array [Array [String "urls",Array [String "b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094",String "b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094",String "b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094"]],Array [String "broker_ca",String "/data/kafka_certs/MSKPOCKey.pem"],Array [String "ssl_cert",String "/data/kafka_devcerts/kafka/signed-certificate-from-acm"],Array [String "ssl_key",String "/data/kafka_devcerts/kafka/key.pem"],Array [String "ssl_key_pwd",String "!{Itbo2aix.D3x7/Yp}!"]])

Just (Array [Array [String "odbc_driver",String "/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so"],Array [String "db_node",String "us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com"],Array [String "db_port",String "5439"],Array [String "db_name",String "redshift_hvr"]])

In [4]:
import Data.Aeson
import Data.Text (Text)
import qualified Data.ByteString.Lazy.Char8 as LBS

data MixedElement = ListElem [Text] |  StringElem Text deriving (Show)

instance ToJSON MixedElement where
    toJSON (StringElem str) = toJSON str
    toJSON (ListElem lst)   = toJSON lst


instance FromJSON MixedElement where
    parseJSON (String str) = return $ StringElem str
    parseJSON (Array arr)  = return $ ListElem ["aa"]
    parseJSON _ = fail "Invalid MixedElement JSON"


--let mixedList = [ListElem ["a","b","c","d"], StringElem "a", StringElem "b"]
let mixedList = [StringElem "a", StringElem "b"]


e = encode mixedList

e

decode e :: Maybe Value
decode e :: Maybe MixedElement

"[\"a\",\"b\"]"

Just (Array [String "a",String "b"])

Just (ListElem ["aa"])

In [15]:
data MixedElement = StringElem String | ListElem [MixedElement]  deriving (Show)

a = [StringElem "a", ListElem [StringElem "a"]]
a 
:t a

[StringElem "a",ListElem [StringElem "a"]]

a :: [MixedElement]

In [23]:
decode redshift :: Maybe [[String]]
decode kafka    :: Maybe String

Just [["odbc_driver","/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so"],["db_node","us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com"],["db_port","5439"],["db_name","redshift_hvr"]]

Nothing

In [32]:
import Control.Applicative
import Data.Aeson
import Data.Text (Text)

type Key      =  String 
data Val    = Val  String | Vals [String] deriving (Show)
data KeyValue = KeyValue Key Val deriving (Show)


instance ToJSON Val where
    toJSON (Val str) = toJSON str
    toJSON (Vals lst) = toJSON lst

instance ToJSON KeyValue where
    toJSON (KeyValue str1 val) = toJSON val

kv = [KeyValue "key1" (Vals ["value1.1","valu1.2"]), KeyValue "key1" (Val  "value1.1")]
v = Vals ["value1.1","valu1.2"]

kv1 = KeyValue "key1" (Vals ["value1.1","valu1.2"])
encode kv1

"[\"value1.1\",\"valu1.2\"]"

In [65]:
{-# LANGUAGE OverloadedStrings #-}

import Control.Applicative
import Data.Aeson
import qualified Data.Aeson.Key as AesonKey


data Val = Val String | Vals [String] deriving (Show)
data KeyValue = KeyValue String Val deriving (Show)

instance ToJSON Val where
    toJSON (Val str) = toJSON str
    toJSON (Vals lst) = toJSON lst

instance ToJSON KeyValue where
    toJSON (KeyValue key value) =    ToJSON  key


kv = [ KeyValue "key1" (Vals ["value1.1","value1.2"]), KeyValue "key2" (Val "value2.1")]
encode (KeyValue "key1" (Vals ["value1.1","value1.2"]))


: 

In [54]:
import GHC.Generics

data Person = Person {
      name :: Text
    , age  :: Int
    } deriving (Generic, Show)
    
    
instance ToJSON Person where
    toEncoding = genericToEncoding defaultOptions

instance FromJSON Person
    
encode (Person {name = "Joe", age = 12})    
decode "{\"name\":\"Joe\",\"age\":12}"

"{\"name\":\"Joe\",\"age\":12}"

Just ()

In [80]:
encode ["a","b"]

decode "[\"a\",\"b\"]" :: Maybe [String]

vv = decode "[\"a\",\"b\"]" :: Maybe Value
vv

"[\"a\",\"b\"]"

Just ["a","b"]

Just (Array [String "a",String "b"])

In [22]:
Array [String "a",String "b"]

: 

In [2]:
:!cabal install --lib vector

Resolving dependencies...
Up to date

In [28]:
{-# LANGUAGE OverloadedStrings #-}

import Data.Aeson
import qualified Data.Vector as V

encode ["a","b"]
vv = decode "[\"a\",\"b\"]" :: Maybe Value
vv

unJust   v = case v of
               Just x -> x
               
listFromArray a = case a of
        Array v ->  v
        _ -> V.empty -- Handle other cases (not an array) 

unString v = case v of 
                String x-> x

a2l = map unString . V.toList . listFromArray .unJust  
        
a2l  vv

"[\"a\",\"b\"]"

Just (Array [String "a",String "b"])

["a","b"]

In [98]:
data KV = KV String String | KVs String [String] deriving (Show)

instance ToJSON KV where
    toJSON (KV key value) = object ["type" .= ("KV" :: String), "key" .= key, "value" .= value]
    toJSON (KVs key values) = object ["type" .= ("KVs" :: String), "key" .= key, "values" .= values]

a = KV "A" "B"
ea = encode a 
decode ea :: Maybe Value

Just (Object (fromList [("key",String "A"),("type",String "KV"),("value",String "B")]))

In [250]:
let a = "\/a"

: 

"[[\"urls\",[\"b-2.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092\",\"b-1.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092\",\"b-3.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092\"]],[\"default_topic\",\"{hvr_tbl_name}\"]]"

In [8]:
:!cabal install --lib vector

Resolving dependencies...
Up to date

In [27]:
import Data.Aeson
import qualified Data.Vector as V
import Data.List (intercalate)
import Data.Text (Text, unpack)
import Data.List.Split (splitOn)

valueFromString  s = decode  s :: Maybe Value

unjust :: Maybe a -> a
unjust (Just a) = a


strfromString (String x) = unpack  x

listFromArray =  intercalate ";" .  map strfromString .unArray

unArray :: Value -> [Value]
unArray (Array x )  = V.toList x

unString  [String x , String y] = [unpack  x, unpack y]
unString  [String x , Array y]  = [unpack  x , listFromArray (Array y) ]


In [ ]:
kafkaStr = "[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"

In [38]:
kafkaStr = "[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"
--kafkaStr= "[[\"urls\",[\"10.242.115.68:9092\",\"10.242.115.9:9092\",\"10.242.115.54:9092\"]],[\"default_topic\",\"{hvr_tbl_name}\"]]" 
kafkaStr2 =  (unjust . valueFromString )kafkaStr 
--(map(unString . unArray) . unArray) kafkaStr2

: 

In [16]:
import Data.Aeson
import qualified Data.Vector as V
import Data.List (intercalate)
import Data.Text (Text, unpack)
import Data.List.Split (splitOn)
import qualified Data.ByteString.Lazy.Char8 as LBS

valueFromString :: LBS.ByteString -> Maybe Value
valueFromString  s = decode  s :: Maybe Value

unjust :: Maybe a -> a
unjust (Just a) = a

strfromString :: Value -> String
strfromString (String x) = unpack  x

unArray :: Value -> [Value]
unArray (Array x )  = V.toList x

unString :: [Value] -> [String]
unString  [String x , String y] = [unpack  x, unpack y]
unString  [String x , Array y]  = [unpack  x , listFromArray (Array y) ]

listFromArray    = intercalate ";" . map strfromString . unArray
getKafkaString   = LBS.pack . last . splitOn "="
parseKafkaString = map(unString . unArray) . unArray . unjust . valueFromString 
filterColumns    = filter (\[k, _] -> k `elem` ["urls", "ssl_key"])

let kafkaStr = "JSON=[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"
(parseKafkaString . getKafkaString) kafkaStr

[["urls","b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094;b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094;b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094"],["broker_ca","/data/kafka_certs/MSKPOCKey.pem"],["ssl_cert","/data/kafka_devcerts/kafka/signed-certificate-from-acm"],["ssl_key","/data/kafka_devcerts/kafka/key.pem"],["ssl_key_pwd","!{Itbo2aix.D3x7/Yp}!"]]

In [26]:
data Columns = ColumnA String | ColumnB String


 
filter (\[k, _] -> k `elem` ["a", "b"]) [["a","1"],["b=c","1"],["b=c","1"]]

: 

In [ ]:
[(ColumnA:"a", ColumnB:"b" }

In [ ]:


kafkaStr= "[\"aa\",\"bb\",\"cc\"]"
valueFromString  s = decode  s :: Maybe Value
kafkaStr2 =(unjust . valueFromString )kafkaStr 


kafkaStr2

strfromString (String x) = unpack  x
listFromArray =  intercalate ";" .  map strfromString .  unArray 
listFromArray kafkaStr2



: 

In [59]:
unArray = V.toList . listFromArray 
vvv =  unJust kafkaStr 

listFromArray a = case a of
        Array v ->  v
        _ -> V.empty -- Handle other cases (not an array) 

unArray []      = []  
unArray (x:xs)  =  (V.toList . listFromArray) x : unArray xs


(unArray . V.toList . listFromArray . unJust) vv

Line 6: Use map
Found:
unArray [] = []
unArray (x : xs) = (V.toList . listFromArray) x : unArray xs
Why not:
unArray xs = map (V.toList . listFromArray) xs

[[String "a",String "b"],[String "a",String "b",String "c"]]

In [200]:
import Data.List (intercalate)


let stringList = ["apple", "banana", "orange"]
let separator = ", "
intercalate ";" stringList

"apple;banana;orange"

In [207]:
let a = head stringList
:type  a

a :: forall {a}. IsString a => a

In [37]:
let kafkaStr = "Json=[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"
:t (LBS.pack  kafkaStr)
:t kafkaStr

(LBS.pack kafkaStr) :: ByteString

kafkaStr :: String